Import Libraries

In [1]:
# %matplotlib inline
# %pygame inline

import os
import gymnasium as gym
# import gym
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
# from matplotlib import pyplot as plt

Load environment example from OpenAI's Gymnasium

In [2]:
environment = 'CartPole-v1'
mode = 'rgb_array'
env = gym.make(environment)
env = DummyVecEnv([lambda: env])

# observation, info = env.reset(seed=42)

Understanding the Environment
Important, five key components in RL: Agent, Action, Environment, Obervation, Rewards

In [3]:
# episodes = 10
# for episode in range(1, episodes+1):
    
#     state = env.reset()
#     done = False
#     score = 0

#     while not done:
#         env.render()
#         action = env.action_space.sample()
#         observation, reward, terminated, truncated, info = env.step(action)
#         score += reward

#         if terminated or truncated:
#             observation, info = env.reset()
#             done = True

#     print('Episode:{} Score:{}'.format(episode, score))
# env.close()

Model Training

In [4]:
log_path = os.path.join('Training', 'Logs')
type(env.action_space)

gymnasium.spaces.discrete.Discrete

In [6]:
env = gym.make(environment)
env = DummyVecEnv([lambda: env])
model = PPO(policy='MlpPolicy', env=env, tensorboard_log=log_path, verbose=1)

Using cuda device


In [7]:
model.learn(total_timesteps=20_000)

Logging to Training\Logs\PPO_68
-----------------------------
| time/              |      |
|    fps             | 342  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 340         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008779552 |
|    clip_fraction        | 0.0934      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000877   |
|    learning_rate        | 0.0003      |
|    loss                 | 7.15        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0139     |
|    value_loss           | 50.8        |
-----------------------------------------
--

Save and Reload Model

In [8]:
PPO_PATH = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
model.save(PPO_PATH)

del model

model = PPO.load(PPO_PATH, env=env)

Model Evaluation

In [9]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

d:\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
d:\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:208: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(500.0, 0.0)

Test Model

In [10]:
env.reset()

episodes = 5
for episode in range(1, episodes+1):
    
    observ = env.reset()
    done = False
    score = 0

    while not done:
        # env.render()
        action, _ = model.predict(observ)
        observ, reward, done, information = env.step(action)
        score += reward

        if done:
            env.reset()
            done = True

    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[155.]
Episode:2 Score:[491.]
Episode:3 Score:[405.]
Episode:4 Score:[500.]
Episode:5 Score:[216.]


Adding a callback to the Training Stage

In [50]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

env = gym.make(environment)
env = DummyVecEnv([lambda: env])

In [51]:
save_path = os.path.join('Training', 'Saved Models')

callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=570.0, verbose=1)

# stop_callback = print("stop_callback called")
# turn_callback = print('turn_callback called')
eval_callback = EvalCallback(eval_env=env,
                             callback_on_new_best=callback_on_best,
                             eval_freq=5_000,
                             best_model_save_path=save_path,
                             verbose=1)

In [52]:
save_path

'Training\\Saved Models'

In [53]:
log_path

'Training\\Logs'

In [54]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [55]:
model.learn(total_timesteps=20_000, callback=eval_callback)

Logging to Training\Logs\PPO_77
-----------------------------
| time/              |      |
|    fps             | 444  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 322         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008398766 |
|    clip_fraction        | 0.0864      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00302     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.34        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0133     |
|    value_loss           | 54.2        |
-----------------------------------------
Ev

In [56]:
import sys
print(sys.getrecursionlimit())

3000


Vieweing Logs in Tensorboard

In [57]:
training_log_path = os.path.join(log_path, 'PPO_2')
training_log_path

# !tensorboard --logdir={training_log_path}

'Training\\Logs\\PPO_2'